#### 1. Use the Notebook to build the code to scrape the Wikipedia page, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe:

In [2]:
import pandas as pd
import requests

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response=requests.get(url)
df_trwk=pd.read_html(response.content, header=0)[0]
#df_trwk.head()

In [3]:
df_trwk.shape

(289, 3)

In [4]:
df_trwk2=df_trwk[df_trwk.Borough != 'Not assigned']
df_trwk2.reset_index(drop=True, inplace=True)
#df_trwk2.head(10)

In [5]:
df_trwk2.shape

(212, 3)

In [5]:
df_trwk2['Neighbourhood'].replace('Not assigned', df_trwk2['Borough'],inplace=True)
#df_trwk2.head(12)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:5886: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [26]:
df_trcom = (df_trwk2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '
            .join(set(x.dropna()))).reset_index())
#df_trcom.head(10)

In [8]:
df_trcom.shape

(103, 3)

#### 2. Use the Geospatial_data csv file to create the merging dataframe:

##### 2-1. Read Geospatial_data csv file

In [8]:
import numpy as np

In [9]:
df_cocln = pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
df_cocln.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
df_cocln.shape

(103, 3)

##### 2-2. Create a merging file

In [22]:
#-- Solutions --
#(op1) out = df1.merge(df2, left_on='store', right_on='store_code')[['id', 'store', 'address', 'warehouse']]
#(op2) out = pd.concat([df1.sort_values('store'),
#                      df2.sort_values('store_code')[['warehouse']].reset_index(drop=1)], 1)
#(op3) s = df1.store.replace(df2.set_index('store_code')['warehouse'])
#(map) df1['warehouse'] = df1['store'].map(df2.set_index('store_code')['warehouse'])
#(join) df1 = df1.join(df2.set_index('store_code'), on=['store']).drop('serialNo', 1)

df_coclm=df_cocln.merge(df_trcom, left_on='Postal Code', right_on='Postcode')\
                       [['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]
df_coclm.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [23]:
df_coclm.tail()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Richview Gardens, Kingsview Village, St. Phill...",43.688905,-79.554724
101,M9V,Etobicoke,"Mount Olive, South Steeles, Thistletown, Silve...",43.739416,-79.588437
102,M9W,Etobicoke,Northwest,43.706748,-79.594054


In [24]:
df_coclm.shape

(103, 5)

##### ** End of work **